In [1]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("../DATA/CONCAT/luft_vi_2023_2024.csv")
df.columns

In [3]:
# Create a pivot table
df['s_status'] = df['s_status'].astype(str)
pivot_table = df.pivot_table(index='R_Sta_txt',values='R_Sta', columns='s_status',  aggfunc='count', fill_value=0)#,values='R_Sta',
pivot_table

s_status,1.0,101.0,102.0,103.0,106.0,107.0,2.0,201.0,202.0,203.0,206.0,207.0,3.0,6.0,7.0,9.0,nan
R_Sta_txt,,,,,,,,,,,,,,,,,
Chemically wet,75770,185,93,54,243,46,52732,98,9,0,1128,18,16464,286,41,174,23888
Critically wet,17,0,0,73,37,21,25,0,0,0,32,9,1713,45,13,176,75
Damp,146872,353,75,4,258,0,54319,156,27,3,1595,9,2454,516,9,6,45179
Dry,256261,510,23,11,353,0,25076,132,38,11,1597,1,1571,736,0,0,47696
Ice,87,0,0,11,615,71,50,0,0,0,311,26,1384,997,35,726,166
Snow / Ice,8,0,0,0,139,4,2,0,0,0,174,1,44,113,2,70,21
Wet,24893,60,74,110,182,54,44071,11,26,11,525,91,84597,204,31,33,23917


In [ ]:
df['R_Sta_txt'].value_counts()

In [ ]:
df['s_status_txt'].value_counts()

In [ ]:
for i in df['s_status_txt'].unique():
    for j in df['R_Sta_txt'].unique():
        print("s_status_txt :",i,":", "R_Sta_txt :",j ,end=": ")
        print(len(df[(df['s_status_txt'] == f"{i}") & (df['R_Sta_txt'] == f"{j}")]))

In [ ]:
for j in df['R_Sta_txt'].unique():
    for i in df['s_status_txt'].unique():
        print("R_Sta_txt :",j, ":","s_status_txt :",i, end=": ")
        print(len(df[(df['s_status_txt'] == f"{i}") & (df['R_Sta_txt'] == f"{j}")]))

# 데이터 시각화

In [5]:
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
import seaborn as sns

plt.rcParams['font.family'] = 'Malgun Gothic' # font 설정
matplotlib.rcParams['axes.unicode_minus'] = False # '-' 부호 인식 설정

In [6]:
df = pd.read_csv("../OUTPUT/aws_sensor.csv")

In [ ]:
df['s_status_txt'].unique()

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import numpy as np

weather = ['TA', 'HM', 'PA']
seansor_data = ['ta1', 'rh1', 'pressure']
vi_road_list = df['s_status_txt'].unique()
# 'Chemically wet', 'Damp', 'Wet', 'Dry', 'Ice', 'Critically wet',  'Snow / Ice', nan
df = df[df['TA'] >= -100]
df = df[df['pressure'] >= 960]
df = df[df['PA'] >= 9600]
# Define the y-axis limits for each plot. Adjust these limits as needed.
y_limits = {
    'ta1': (-10, 40),  
    'rh1': (0, 100),  
    'pressure': (900, 1050), 
    'PA': (950, 1050),
    'TA': (-10, 40),   
    'HM': (0, 100)     
}

# Define the order and colors for the legend
hue_order = [ 'Dry','Damp', 'Wet', 'Snow / Ice','Ice','Chemically wet',  'Critically wet']# , np.nan] 
# palette = {
#     'Dry': 'blue',
#     'Moist': 'green',
#     'Wet': 'cyan',
#     'Snow': 'white',
#     'Ice': 'gray',
#     'Slush': 'yellow',
#     'error': 'red',
#     # np.nan: 'black'  # Assign a color for NaN if necessary
# }

palette = {
    'Dry': 'blue',
    'Chemically wet': 'green',
    'Wet': 'cyan',
    'Snow / Ice': 'white',
    'Ice': 'gray',
    'Critically wet': 'orange',
    'Damp': 'yellow',
    # np.nan: 'black'  # Assign a color for NaN if necessary
}




# Mapping of sensor data to descriptive names
x_labels = {
    'ta1': 'Temperature (°C)',
    'rh1': 'Humidity (%)',
    'pressure': 'Pressure (hPa)',
    'TA': 'Temperature (°C)',
    'HM': 'Humidity (%)',
    'PA': 'Pressure (hPa)'
}

for i in vi_road_list:
    for j in range(len(weather)):
        elem2 = weather[j]
        elem1 = seansor_data[j]
        df_graph1 = df[df['s_status_txt'] == i]
        df_graph = df_graph1.copy()
        df_graph["HM"] = df_graph["HM"] / 10
        df_graph["TA"] = df_graph["TA"] / 10
        df_graph["PA"] = df_graph["PA"] / 10
        
        # Create a gridspec layout with specified width ratios
        fig = plt.figure(figsize=(18, 6))
        gs = gridspec.GridSpec(1, 3, width_ratios=[1, 5, 1])  # Adjust the width_ratios as needed
        
        # Create the first box plot on the left
        ax0 = plt.subplot(gs[0])
        sns.boxplot(data=df_graph, y=elem1, ax=ax0)
        ax0.set_title(f'바이살라 \n{i}')
        ax0.spines['top'].set_visible(False)
        ax0.spines['right'].set_visible(False)
        ax0.spines['left'].set_visible(False)
        ax0.set_ylim(y_limits[elem1])  # Set y-axis limits for the first plot
        ax0.set_xlabel(x_labels[elem1])  # Add x-axis label
        
        # Create the second box plot in the middle with hue
        ax1 = plt.subplot(gs[1])
        sns.boxplot(data=df_graph, y=elem1, hue='R_Sta_txt', ax=ax1, hue_order=hue_order, palette=palette)
        ax1.set_title(f'루프트')
        ax1.legend(title='Status', bbox_to_anchor=(0.9, 1), loc='upper left', fontsize='large')
        ax1.spines['top'].set_visible(False)
        ax1.spines['right'].set_visible(False)
        ax1.spines['left'].set_visible(False)
        ax1.set_ylim(y_limits[elem1])  # Set y-axis limits for the second plot
        ax1.set_xlabel(x_labels[elem1])  # Add x-axis label
        
        # Create the third box plot on the right
        ax2 = plt.subplot(gs[2])
        sns.boxplot(data=df_graph, y=elem2, ax=ax2)
        ax2.set_title(f'aws')
        ax2.spines['top'].set_visible(False)
        ax2.spines['right'].set_visible(False)
        ax2.spines['left'].set_visible(False)
        ax2.set_ylim(y_limits[elem2])  # Set y-axis limits for the third plot
        ax2.set_xlabel(x_labels[elem2])  # Add x-axis label

        
        # Display the plots      
        plt.tight_layout()
        name = f"{elem1}_{elem2}_{i}".replace("/", "_")  # Replacing / with _ in the file name
        # name = f"{elem1}_{elem2}_{i}".replace("", "_")  # Replacing / with _ in the file name
        plt.savefig(f'../OUTPUT/IMG/{name}.jpeg', dpi = 500)
        plt.clf()
